<a href="https://colab.research.google.com/github/Tommy-Las/WatfordFC/blob/main/Data_cleaning_transformation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# If google colab

In [ ]:
# Mount drive

from google.colab import drive
drive.mount('/content/drive')

# Import packages and data

In [1]:
import pandas as pd
# Turn off warnings
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', None)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder

Import GPS data

In [2]:
# file_path = '/content/drive/MyDrive/WATFORD FC/Datos GPS/GPS 2018-2023.xlsx'
file_path = 'data/GPS 2018-2023_NoContact.xlsx'
df_gps = pd.read_excel(file_path)

Import speed data

In [3]:
# file_path = '/content/drive/MyDrive/WATFORD FC/Datos GPS/max_speed.xlsx'
file_path = 'data/max_speed.xlsx'
df_speed = pd.read_excel(file_path)

Import wellbeing data


In [6]:
#file_path = '/content/drive/MyDrive/WATFORD FC/Datos Wellbeing/wellbeing ssp.xlsx'
#df_wellbeing = pd.read_excel(file_path)

# Merge Sprint rows into a single row per session

The sprint values are separated in different rows per session ,

We want to merge in into a single row per session.

## Transform data before merging sprint rows

In [4]:
df_speed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123180 entries, 0 to 123179
Data columns (total 11 columns):
 #   Column                                       Non-Null Count   Dtype  
---  ------                                       --------------   -----  
 0   DATE                                         123180 non-null  object 
 1   ID                                           123180 non-null  int64  
 2   Player Position                              123155 non-null  object 
 3   Max Speed                                    116475 non-null  float64
 4   Sprints                                      113560 non-null  float64
 5   MINUTES                                      113654 non-null  float64
 6   Season                                       123180 non-null  object 
 7   Max Speed Season                             123180 non-null  float64
 8   Avg Speed Season                             122357 non-null  float64
 9   % Max Speed                                  122357 non-nul

We want to remove rows after 08-07-2021 since we don't have data for sprints and speed

In [5]:
from datetime import date
# Convert DATE column values into Pandas datetime object
df_speed['DATE'] = pd.to_datetime(df_speed['DATE'], dayfirst=True)

# Filter rows after date: '2021-08-07'
date_filter = pd.Timestamp('2021-08-07')
df_speed = df_speed[df_speed['DATE'] > date_filter]

# Filter rows after date: '2023-05-11'
date_filter = pd.Timestamp('2023-05-11')
df_speed = df_speed[df_speed['DATE'] < date_filter]

date_filter = pd.Timestamp('2021-06-28')
df_gps = df_gps[df_gps['DATE'] > date_filter]

In [6]:
df_speed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 55390 entries, 58257 to 113646
Data columns (total 11 columns):
 #   Column                                       Non-Null Count  Dtype         
---  ------                                       --------------  -----         
 0   DATE                                         55390 non-null  datetime64[ns]
 1   ID                                           55390 non-null  int64         
 2   Player Position                              55365 non-null  object        
 3   Max Speed                                    55377 non-null  float64       
 4   Sprints                                      55362 non-null  float64       
 5   MINUTES                                      55389 non-null  float64       
 6   Season                                       55390 non-null  object        
 7   Max Speed Season                             55390 non-null  float64       
 8   Avg Speed Season                             55390 non-null  float64       


Replace NULL sprint values to 0

In [7]:
# Replace NULL values from Sprint to 0
df_speed['Sprints'] = df_speed['Sprints'].fillna(0)

## Merge sprint values

Print values to compare merge later

In [8]:
df_speed.sort_values(by='DATE', ascending=False).head(15)

,DATE,ID,Player Position,Max Speed,Sprints,MINUTES,Season,Max Speed Season,Avg Speed Season,% Max Speed,%Speed diference against max. Speed average
58257,2023-05-10,23085,CENTRE MIDFIELDER,22.91,3.0,30.19,2022-2023,32.12,20.304388,71.326276,12.832753
58258,2023-05-10,87583,CENTRE MIDFIELDER,24.70,20.0,40.53,2022-2023,34.27,21.189939,72.074701,16.564751
58259,2023-05-09,96811,CENTRE MIDFIELDER,32.19,10.0,70.51,2022-2023,36.04,20.882454,89.317425,54.148548
58260,2023-05-09,89871,STRIKER,25.46,0.0,79.87,2022-2023,34.12,21.753402,74.618992,17.039162
58300,2023-05-08,55555,FULL BACK,30.47,18.0,72.62,2022-2023,34.94,22.071250,87.206640,38.052897
58289,2023-05-08,48692,CENTRE BACK,32.56,35.0,69.15,2022-2023,36.21,21.029270,89.919912,54.831811
58290,2023-05-08,87583,CENTRE MIDFIELDER,22.69,11.0,34.13,2022-2023,34.27,21.189939,66.209513,7.079117
58291,2023-05-08,37537,CENTRE MIDFIELDER,28.65,5.0,47.05,2022-2023,34.09,21.974456,84.042241,30.378655
58292,2023-05-08,37537,CENTRE MIDFIELDER,30.59,3.0,22.10,2022-2023,34.09,21.974456,89.733060,39.207088
58293,2023-05-08,37537,CENTRE MIDFIELDER,30.59,8.0,69.15,2022-2023,34.09,21.974456,89.733060,39.207088


Do a group by player and date, and select the maximum value for each feature

In [9]:
df_speed = df_speed.groupby(['DATE', 'ID']).agg('max').reset_index()

In [10]:
df_speed.sort_values(by='DATE', ascending=False).head(15)

,DATE,ID,Player Position,Max Speed,Sprints,MINUTES,Season,Max Speed Season,Avg Speed Season,% Max Speed,%Speed diference against max. Speed average
8262,2023-05-10,87583,CENTRE MIDFIELDER,24.70,20.0,40.53,2022-2023,34.27,21.189939,72.074701,16.564751
8261,2023-05-10,23085,CENTRE MIDFIELDER,22.91,3.0,30.19,2022-2023,32.12,20.304388,71.326276,12.832753
8260,2023-05-09,96811,CENTRE MIDFIELDER,32.19,10.0,70.51,2022-2023,36.04,20.882454,89.317425,54.148548
8259,2023-05-09,89871,STRIKER,25.46,0.0,79.87,2022-2023,34.12,21.753402,74.618992,17.039162
8249,2023-05-08,55555,FULL BACK,30.47,18.0,72.62,2022-2023,34.94,22.071250,87.206640,38.052897
8240,2023-05-08,10103,FULL BACK,21.24,1.0,44.84,2022-2023,34.95,22.202230,60.772532,-4.333932
8241,2023-05-08,12086,WINGER,35.04,41.0,88.27,2022-2023,35.44,21.901184,98.871332,59.991349
8242,2023-05-08,19817,FULL BACK,31.68,0.0,128.04,2022-2023,35.77,21.605460,88.565837,46.629599
8243,2023-05-08,21079,WINGER,34.51,51.0,88.04,2022-2023,36.97,22.708654,93.345956,51.968499
8244,2023-05-08,23085,CENTRE MIDFIELDER,23.71,7.0,58.52,2022-2023,32.12,20.304388,73.816936,16.772788


# Handle duplicates for GPS Data

There are some players that have 2 records for the same day, they are different amounts so we sum them

In [11]:
# Define columns to sum
columns_to_sum = ['Total D', '>19.8', '> 25 Km/h', 'ACC', 'DEC']

# Define columns to select the first value
columns_to_first = ['DATE', 'Column2', 'PLAYER', 'Injury', 'season', 'LEAGUE', 'preseason-season', 'MANAGER']

# Group by the duplicate subset and aggregate
df_gps_aggregated = (
    df_gps[df_gps.duplicated(subset=['PLAYER', 'DATE'], keep=False)]
    .groupby(['PLAYER', 'DATE'], as_index=False)
    .agg({**{col: 'sum' for col in columns_to_sum},
          **{col: 'first' for col in columns_to_first}})
)

# Ensure non-duplicated rows are preserved by combining them back
df_gps_combined = pd.concat([
    df_gps[~df_gps.duplicated(subset=['PLAYER', 'DATE'], keep=False)],
    df_gps_aggregated
], ignore_index=True)

# Merge GPS and Speed dataframes

## Prepare the data before the merge

Verify column names for both Dataframes

In [15]:
df_gps_combined.columns

Index(['DATE', 'Column2', 'PLAYER', 'Injury', 'season', 'LEAGUE',
       'preseason-season', 'MANAGER', 'Total D', '>19.8', '> 25 Km/h', 'ACC',
       'DEC'],
      dtype='object')

In [16]:
df_speed.columns

Index(['DATE', 'ID', 'Player Position', 'Max Speed', 'Sprints', 'MINUTES',
       'Season', 'Max Speed Season', 'Avg Speed Season', '% Max Speed',
       '%Speed diference against max. Speed average'],
      dtype='object')

Change datatypes before the merge

In [17]:
# Convert DATE column from GPS df into datetime pandas obkect
df_gps_combined['DATE'] = pd.to_datetime(df_gps_combined['DATE'], dayfirst=True)

# Drop NULL values for 'PLAYER'
df_gps_combined = df_gps_combined.dropna(subset=['PLAYER'])

# Convert ID and PLAYER columns to the same data type - integers
df_gps_combined['PLAYER'] = df_gps_combined['PLAYER'].astype(int)
df_speed['ID'] = df_speed['ID'].astype(int)

## Do the merge of gps & speed dataframes

In [12]:
# Perform an inner join on matching DATE and PLAYER/ID values
df_merged = df_gps_combined.merge(df_speed, left_on=['DATE', 'PLAYER'], right_on=['DATE', 'ID'], how='inner')

In [13]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7835 entries, 0 to 7834
Data columns (total 23 columns):
 #   Column                                       Non-Null Count  Dtype         
---  ------                                       --------------  -----         
 0   DATE                                         7835 non-null   datetime64[ns]
 1   Column2                                      7835 non-null   object        
 2   PLAYER                                       7835 non-null   float64       
 3   Injury                                       7835 non-null   float64       
 4   season                                       7835 non-null   object        
 5   LEAGUE                                       7835 non-null   object        
 6   preseason-season                             7835 non-null   object        
 7   MANAGER                                      7835 non-null   object        
 8   Total D                                      7835 non-null   object        
 9

Verify class imbalances

In [14]:
print("No Injury: " + str(df_merged[df_merged["Injury"] == 0].shape[0]))
print("Injury: " + str(df_merged[df_merged["Injury"] == 1].shape[0]))
print(f"%: {((df_merged[df_merged['Injury'] == 1].shape[0] / df_merged.shape[0]) * 100):.2f}%")

No Injury: 7798
Injury: 37
%: 0.47%


# Merge df with 'weight' in wellbeing dataframe

In [ ]:
# df_wellbeing = df_wellbeing[["Weight", "Date", "PLAYER"]]
# df_wellbeing.info()

In [ ]:
# # Sort by PLAYER and DATE to ensure correct order for backfilling
# df_wellbeing = df_wellbeing.sort_values(by=['PLAYER', 'Date'])

# # Fill the NULL values in 'Weight' with the previous valid value for each player
# df_wellbeing['Weight'] = df_wellbeing.groupby('PLAYER')['Weight'].bfill()
# df_wellbeing['Weight'] = df_wellbeing.groupby('PLAYER')['Weight'].ffill()

# df_wellbeing.info()

In [ ]:
# # Merge the DataFrames
# df_merged = df_merged.merge(df_wellbeing, left_on=['DATE', 'PLAYER'], right_on=['Date', 'PLAYER'], how='left')

# # Sort by PLAYER and DATE to ensure chronological order
# df_merged = df_merged.sort_values(by=['PLAYER', 'DATE'])

# # Back-fill null values in 'Weight' for each PLAYER
# df_merged['Weight'] = df_merged.groupby('PLAYER')['Weight'].bfill()
# df_merged['Weight'] = df_merged.groupby('PLAYER')['Weight'].ffill()

# # Optional: Drop the 'Date' column from the right DataFrame if not needed
# df_merged = df_merged.drop(columns=['Date'])

# # Display the final DataFrame
# df_merged.info()

In [ ]:
#df_merged.groupby('PLAYER')['Weight'].apply(lambda x: x.isnull().sum())

# Transform numeric columns to the same data types

In [15]:
cols = ['Total D', '>19.8', '> 25 Km/h', 'ACC',
       'DEC', 'ID', 'Max Speed', 'Sprints', 'MINUTES', 'Max Speed Season',
       'Avg Speed Season', '% Max Speed',
       '%Speed diference against max. Speed average']

df_merged[cols] = df_merged[cols].astype(float)

In [16]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7835 entries, 0 to 7834
Data columns (total 23 columns):
 #   Column                                       Non-Null Count  Dtype         
---  ------                                       --------------  -----         
 0   DATE                                         7835 non-null   datetime64[ns]
 1   Column2                                      7835 non-null   object        
 2   PLAYER                                       7835 non-null   float64       
 3   Injury                                       7835 non-null   float64       
 4   season                                       7835 non-null   object        
 5   LEAGUE                                       7835 non-null   object        
 6   preseason-season                             7835 non-null   object        
 7   MANAGER                                      7835 non-null   object        
 8   Total D                                      7835 non-null   float64       
 9

# Values that are 0 for ACC, DEC, Max Speed

Perform imputations on missing data

In [17]:
# Count of rows containing 0 for each column
zero_counts = (df_merged == 0).sum()

# Creating a DataFrame for the results
result_df = pd.DataFrame({
    "Column Name": zero_counts.index,
    "Number of Rows with 0": zero_counts.values
})

result_df

,Column Name,Number of Rows with 0
0,DATE,0
1,Column2,382
2,PLAYER,0
3,Injury,7798
4,season,0
5,LEAGUE,0
6,preseason-season,0
7,MANAGER,0
8,Total D,3
9,>19.8,420


Perform KNN Imputation

In [18]:
# Columns to impute
metrics_imputation_zeroes = ['Total D', 'ACC', 'DEC', 'Max Speed', 'MINUTES']

# Columns to use for KNN calculation
columns_for_knn = ['Total D', '>19.8', '> 25 Km/h', 'ACC', 'DEC', 'ID',
                   'Max Speed', 'Sprints', 'MINUTES', '% Max Speed']

# Replace 0 with NaN in the columns to be imputed
df_merged[metrics_imputation_zeroes] = df_merged[metrics_imputation_zeroes].replace(0, np.nan)

# Ensure the columns used for KNN calculation are numeric
columns_for_knn_numeric = df_merged[columns_for_knn].select_dtypes(include=[np.number]).columns.tolist()

# Initialize the KNNImputer
knn_imputer = KNNImputer(n_neighbors=5)  # Adjust n_neighbors if needed

# Perform KNN imputation using the broader set of columns for calculation
imputed_values = knn_imputer.fit_transform(df_merged[columns_for_knn_numeric])

# Update only the specified columns to impute
df_merged[metrics_imputation_zeroes] = imputed_values[:,
    [columns_for_knn_numeric.index(col) for col in metrics_imputation_zeroes]]

In [19]:
# Count of rows containing 0 for each column
zero_counts = (df_merged == 0).sum()

# Creating a DataFrame for the results
result_df = pd.DataFrame({
    "Column Name": zero_counts.index,
    "Number of Rows with 0": zero_counts.values
})

result_df

,Column Name,Number of Rows with 0
0,DATE,0
1,Column2,382
2,PLAYER,0
3,Injury,7798
4,season,0
5,LEAGUE,0
6,preseason-season,0
7,MANAGER,0
8,Total D,0
9,>19.8,420


Calculate '% Max Speed' and '% Speed difference against max. Speed average'

In [20]:
# Calculate '% Max Speed' for all rows
df_merged['% Max Speed'] = (df_merged['Max Speed'] / df_merged['Max Speed Season']) * 100

# Calculate '% Speed difference against max. Speed average' for all rows
df_merged['%Speed diference against max. Speed average'] = (
    (df_merged['Max Speed'] - df_merged['Avg Speed Season']) / df_merged['Avg Speed Season']
) * 100

# Reduce data volume/size

## Players who have never been injured

We are going to filter by players that had at least one injury, to reduce # of rows and hopefully improve accuracy

In [21]:
# Group by ID and sum the Injury column
injury_counts = df_merged.groupby('ID').agg({'Injury': 'sum'})

# Get the IDs where the sum of Injury is greater than 0
ids_to_remove = injury_counts[injury_counts['Injury'] == 0].index.tolist()

# Remove these IDs from the original DataFrame
df_filtered = df_merged[~df_merged['ID'].isin(ids_to_remove)]

In [22]:
print("No Injury: " + str(df_filtered[df_filtered["Injury"] == 0].shape[0]))
print("Injury: " + str(df_filtered[df_filtered["Injury"] == 1].shape[0]))
print(f"%: {((df_filtered[df_filtered['Injury'] == 1].shape[0] / df_filtered.shape[0]) * 100):.2f}%")

No Injury: 4686
Injury: 37
%: 0.78%


Verify that at least one player had an injury

In [23]:
df_filtered.groupby('ID').agg({'Injury': 'sum'})

,Injury
ID,
10103.0,4.0
10452.0,2.0
12086.0,1.0
17316.0,2.0
18096.0,3.0
20083.0,1.0
21079.0,2.0
23081.0,2.0
23085.0,2.0


# Change column names & drop unnecesary columns

In [25]:
column_rename_dict = {
    'Column2': 'Microcycle',
    'DATE': 'DATE',
    'ID': 'PlayerID',
    'Total D': 'TD',
    '>19.8': '>19.8',
    '> 25 Km/h': '>25',
    'ACC': 'ACC',
    'DEC': 'DEC',
    'Max Speed': 'Max Speed',
    'Max Speed Season': 'Max Speed Season',
    'Avg Speed Season': 'Avg Speed Season',
    '% Max Speed': '% Max Speed',
    '%Speed diference against max. Speed average': 'Speed Diff Max Avg',
    'Injury': 'Injury',
    'MINUTES': 'Mins',
    'Sprints': 'Sprints',
    'Total D_Rel': 'TD_Rel',
    '>19.8_Rel': '>19.8_Rel',
    '> 25 Km/h_Rel': '>25 Km/h_Rel',
    'ACC_Rel': 'ACC_Rel',
    'DEC_Rel': 'DEC_Rel',
    'Sprints_Rel': 'Sprints_Rel'
}

df_filtered.rename(columns=column_rename_dict, inplace=True)

df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4723 entries, 0 to 7802
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   DATE                4723 non-null   datetime64[ns]
 1   Microcycle          4723 non-null   object        
 2   PLAYER              4723 non-null   float64       
 3   Injury              4723 non-null   float64       
 4   season              4723 non-null   object        
 5   LEAGUE              4723 non-null   object        
 6   preseason-season    4723 non-null   object        
 7   MANAGER             4723 non-null   object        
 8   TD                  4723 non-null   float64       
 9   >19.8               4723 non-null   float64       
 10  >25                 4723 non-null   float64       
 11  ACC                 4723 non-null   float64       
 12  DEC                 4723 non-null   float64       
 13  PlayerID            4723 non-null   float64       
 1

In [26]:
df_filtered = df_filtered.drop(columns=['LEAGUE', 'MANAGER', 'PLAYER', 'Season'])

# Calculate relative values

Remove spaces in 'Microcycle' column

In [27]:
df_filtered['Microcycle'] = df_filtered['Microcycle'].str.replace(' ', '', regex=False)
df_filtered["Microcycle"].unique()

array(['M+2', 'M+1', 'MD', 'M-1', 'M-2', 'M-3', 'M-5', 'M+3', 'M-4',
       'MD(AWAY)', 'MD(HOME)', nan], dtype=object)

To calculate the relative values, we want the max and avg values for players that played over 85 in a Match Day

In [28]:
def calculate_relative_values(df_original, metrics):
    # Array that holds match days
    microcycle_values = ['MD', 'MD(HOME)', 'MD(AWAY)']

    # Do a copy of the original DF
    df_copy = df_original.copy()

    # Filter players with +85 mins and match day
    df_filtered_85_md = df_copy[
        (df_copy['Mins'] > 85) &
        (df_copy['Microcycle'].isin(microcycle_values))
    ]

    # Players who never completed 85 mins in matchday
    df_filtered_85_only_mins = df_copy[df_copy['Mins'] > 85]

    # Combine both DataFrames
    df_filtered_85 = pd.concat([df_filtered_85_md, df_filtered_85_only_mins])

    # Temporary dictionaries to hold max and avg values per player
    player_max = {}
    player_avg = {}

    # Calculate max and avg for each player
    for player in df_filtered_85['PlayerID'].unique():
        player_data = df_filtered_85[df_filtered_85['PlayerID'] == player]
        player_max[player] = player_data[metrics].max()
        player_avg[player] = player_data[metrics].mean()

    # Add relative values to the original DataFrame
    for metric in metrics:

        df_original[f"{metric}_Rel"] = np.nan

        for player in df_original['PlayerID'].unique():
            if player in player_max and player in player_avg:
                max_value = player_max[player][metric]
                avg_value = player_avg[player][metric]

                # Compute relative value
                df_original.loc[df_original['PlayerID'] == player, f"{metric}_Rel"] = (
                    (df_original.loc[df_original['PlayerID'] == player, metric] * 100) /
                    ((max_value + avg_value) / 2)
                ).round(2)

    return df_original

Call function to calculate relative values

In [29]:
metrics_rel = ['TD', 'HSR', '+25 Km/h', 'ACC', 'DEC']

# Avoid for now
#df_rel = calculate_relative_values(df_filtered, metrics_rel)

df_rel = df_filtered.copy()

# Function for densities

In [ ]:
# cols_mins = ['TD', 'HSR', '+25 Km/h', 'ACC', 'DEC', 'Sprints']

# # Create new columns by dividing by the 'Minutes' column
# for col in cols_mins:
#     df_rel[f'{col}/Mins'] = df_rel[col] / df_rel['Mins']

In [ ]:
# df_rel['Sprints/Mins'] = df_rel['Sprints/Mins'].fillna(0)

# Player loads

## Calculate loads

In [30]:
def calcular_acumulado(df, columnas_calcular, dias):
    # Create an empty list to store processed player DataFrames
    processed_players = []

    # Process each player separately
    for player_id in df['PlayerID'].unique():
        # Filter data for the current player
        player_data = df[df['PlayerID'] == player_id].copy()

        # Create a full date range for the player (from the first to the last recorded date)
        full_date_range = pd.date_range(start=player_data['DATE'].min(), end=player_data['DATE'].max(), freq='D')

        # Set 'DATE' as the index and reindex to fill missing dates with zeros
        player_data = player_data.set_index('DATE').reindex(full_date_range, fill_value=0).reset_index()
        player_data.rename(columns={'index': 'DATE'}, inplace=True)
        player_data['PlayerID'] = player_id

        # Perform rolling calculations for each metric, excluding the current day
        for dia in dias:
            for col in columnas_calcular:
                # Check if the column exists to avoid errors
                if col in player_data.columns:
                    # Rolling sum, mean, std: exclude the current day using shift(1)
                    player_data[f'{col}-{dia}'] = (
                        player_data[col].shift(1).rolling(window=dia, min_periods=1).sum()
                    )
                    # player_data[f'{col}-{dia}-avg'] = (
                    #     player_data[col].rolling(window=dia, min_periods=1).mean()
                    # )
                    # player_data[f'{col}-{dia}-std'] = (
                    #     player_data[col].rolling(window=dia, min_periods=1).std()
                    # )

                    # # EWMA: Exclude the current day using shift(1)
                    # player_data[f'{col}_EWMA-{dia}'] = (
                    #     player_data[col].ewm(span=dia, adjust=False).mean()
                    # )

        # Drop rows where all calculated values are zero (rest days)
        mask_non_zero = (player_data[columnas_calcular].sum(axis=1) > 0)
        player_data = player_data[mask_non_zero]

        # Append the processed player's data to the list
        processed_players.append(player_data)

    # Concatenate all processed player DataFrames into a single DataFrame
    df_resultado = pd.concat(processed_players, ignore_index=True)

    return df_resultado


In [31]:
cols_calculate = ['TD', '>19.8', '>25', 'ACC', 'DEC', 'Sprints', 'Mins']

cumulative_df = calcular_acumulado(df_rel, cols_calculate, [1,3,7,21]) # loads -1, -3, -7, -21

In [32]:
cumulative_df.head()

,DATE,Microcycle,Injury,season,preseason-season,TD,>19.8,>25,ACC,DEC,PlayerID,Player Position,Max Speed,Sprints,Mins,Max Speed Season,Avg Speed Season,% Max Speed,Speed Diff Max Avg,TD-1,>19.8-1,>25-1,ACC-1,DEC-1,Sprints-1,Mins-1,TD-3,>19.8-3,>25-3,ACC-3,DEC-3,Sprints-3,Mins-3,TD-7,>19.8-7,>25-7,ACC-7,DEC-7,Sprints-7,Mins-7,TD-21,>19.8-21,>25-21,ACC-21,DEC-21,Sprints-21,Mins-21
0,2021-08-10,M-4,0.0,2021-2022,SEASON,5651.0,197.0,34.0,63.0,36.0,23085.0,CENTRE MIDFIELDER,28.41,19.0,80.35,30.73,20.278653,92.450374,40.098060,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-08-11,M-3,0.0,2021-2022,SEASON,7423.0,426.0,20.0,52.0,38.0,23085.0,CENTRE MIDFIELDER,26.79,24.0,90.85,30.73,20.278653,87.178653,32.109364,5651.0,197.0,34.0,63.0,36.0,19.0,80.35,5651.0,197.0,34.0,63.0,36.0,19.0,80.35,5651.0,197.0,34.0,63.0,36.0,19.0,80.35,5651.0,197.0,34.0,63.0,36.0,19.0,80.35
2,2021-08-12,M-2,0.0,2021-2022,SEASON,1268.0,0.0,0.0,11.0,5.0,23085.0,CENTRE MIDFIELDER,16.57,0.0,43.58,30.73,20.278653,53.921250,-18.288460,7423.0,426.0,20.0,52.0,38.0,24.0,90.85,13074.0,623.0,54.0,115.0,74.0,43.0,171.20,13074.0,623.0,54.0,115.0,74.0,43.0,171.20,13074.0,623.0,54.0,115.0,74.0,43.0,171.20
3,2021-08-13,M-1,0.0,2021-2022,SEASON,4135.0,112.0,2.0,47.0,20.0,23085.0,CENTRE MIDFIELDER,25.47,11.0,58.35,30.73,20.278653,82.883176,25.600056,1268.0,0.0,0.0,11.0,5.0,0.0,43.58,14342.0,623.0,54.0,126.0,79.0,43.0,214.78,14342.0,623.0,54.0,126.0,79.0,43.0,214.78,14342.0,623.0,54.0,126.0,79.0,43.0,214.78
4,2021-08-15,M+1,0.0,2021-2022,SEASON,6379.0,86.0,0.0,49.0,48.0,23085.0,CENTRE MIDFIELDER,23.37,13.0,79.48,30.73,20.278653,76.049463,15.244339,0.0,0.0,0.0,0.0,0.0,0.0,0.00,5403.0,112.0,2.0,58.0,25.0,11.0,101.93,18477.0,735.0,56.0,173.0,99.0,54.0,273.13,18477.0,735.0,56.0,173.0,99.0,54.0,273.13


## Calculate different load metrics

ACWR and MSWR values, trying to replicate the research paper

In [33]:
def calculate_metrics_loads(df, metrics):

  # Calculate ACWR, MSWR for each metric
  for metric in metrics:

      # Calculate 7-day and 28-day averages for ACWR
      df[f'{metric}_ACWR'] = df[f'{metric}-7-avg'] / df[f'{metric}-28-avg']

      # Calculate mean and standard deviation for MSWR
      df[f'{metric}_MSWR'] = df[f'{metric}-7-avg'] / df[f'{metric}-7-std']

  return df

In [50]:
# cols_calculate = ['TD', 'HSR', '+25 Km/h', 'ACC', 'DEC']
# cumulative_df = calculate_metrics_loads(cumulative_df, cols_calculate)

In [51]:
# cumulative_df.drop(columns=['Sprints-28-avg', 'Sprints-28-std', 'DEC-28-avg', 'DEC-28-std', 'ACC-28-avg', 'ACC-28-std','+25 Km/h-28-avg', '+25 Km/h-28-std', 'HSR-28-avg', 'HSR-28-std', 'TD-28-avg', 'TD-28-std',
#                             'Sprints-7-avg', 'Sprints-7-std', 'DEC-7-avg', 'DEC-7-std', 'ACC-7-avg', 'ACC-7-std','+25 Km/h-7-avg', '+25 Km/h-7-std', 'HSR-7-avg', 'HSR-7-std', 'TD-7-avg', 'TD-7-std', 'TD-28', 'HSR-28', '+25 Km/h-28', 'ACC-28', 'DEC-28', 'Sprints-28',
#                             '+25 Km/h_EWMA-28', 'Sprints_EWMA-28', 'TD_EWMA-28', 'HSR_EWMA-28', 'ACC_EWMA-28', 'DEC_EWMA-28','Sprints_EWMA-7'] , inplace=True)

## For player loads, drop rows with NULL or 0 values

In [34]:
cumulative_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4723 entries, 0 to 4722
Data columns (total 47 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   DATE                4723 non-null   datetime64[ns]
 1   Microcycle          4508 non-null   object        
 2   Injury              4723 non-null   float64       
 3   season              4723 non-null   object        
 4   preseason-season    4723 non-null   object        
 5   TD                  4723 non-null   float64       
 6   >19.8               4723 non-null   float64       
 7   >25                 4723 non-null   float64       
 8   ACC                 4723 non-null   float64       
 9   DEC                 4723 non-null   float64       
 10  PlayerID            4723 non-null   float64       
 11  Player Position     4723 non-null   object        
 12  Max Speed           4723 non-null   float64       
 13  Sprints             4723 non-null   float64     

Drop rows that contain more than 2 NULL values

In [35]:
# Exclude 'Microcycle' column for counting NULLs
columns_to_check = cumulative_df.drop(columns=['Microcycle']).columns

# Drop rows where 2 or more null values exist (excluding 'Microcycle')
cumulative_df = cumulative_df[cumulative_df[columns_to_check].notna().sum(axis=1) > (len(columns_to_check) - 1)]

# Display the resulting DataFrame
cumulative_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4700 entries, 1 to 4722
Data columns (total 47 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   DATE                4700 non-null   datetime64[ns]
 1   Microcycle          4485 non-null   object        
 2   Injury              4700 non-null   float64       
 3   season              4700 non-null   object        
 4   preseason-season    4700 non-null   object        
 5   TD                  4700 non-null   float64       
 6   >19.8               4700 non-null   float64       
 7   >25                 4700 non-null   float64       
 8   ACC                 4700 non-null   float64       
 9   DEC                 4700 non-null   float64       
 10  PlayerID            4700 non-null   float64       
 11  Player Position     4700 non-null   object        
 12  Max Speed           4700 non-null   float64       
 13  Sprints             4700 non-null   float64       
 1

In [36]:
# Count of rows containing 0 for each column
zero_counts = (cumulative_df == 0).sum()

# Creating a DataFrame for the results
result_df = pd.DataFrame({
    "Column Name": zero_counts.index,
    "Number of Rows with 0": zero_counts.values
})

result_df

,Column Name,Number of Rows with 0
0,DATE,0
1,Microcycle,0
2,Injury,4663
3,season,0
4,preseason-season,0
5,TD,0
6,>19.8,285
7,>25,1612
8,ACC,0
9,DEC,0


Remove rows where 3 or more rows contain 0 for the loads

In [37]:
cumulative_df.columns

Index(['DATE', 'Microcycle', 'Injury', 'season', 'preseason-season', 'TD',
       '>19.8', '>25', 'ACC', 'DEC', 'PlayerID', 'Player Position',
       'Max Speed', 'Sprints', 'Mins', 'Max Speed Season', 'Avg Speed Season',
       '% Max Speed', 'Speed Diff Max Avg', 'TD-1', '>19.8-1', '>25-1',
       'ACC-1', 'DEC-1', 'Sprints-1', 'Mins-1', 'TD-3', '>19.8-3', '>25-3',
       'ACC-3', 'DEC-3', 'Sprints-3', 'Mins-3', 'TD-7', '>19.8-7', '>25-7',
       'ACC-7', 'DEC-7', 'Sprints-7', 'Mins-7', 'TD-21', '>19.8-21', '>25-21',
       'ACC-21', 'DEC-21', 'Sprints-21', 'Mins-21'],
      dtype='object')

In [38]:
print("No Injury: " + str(cumulative_df[cumulative_df["Injury"] == 0].shape[0]))
print("Injury: " + str(cumulative_df[cumulative_df["Injury"] == 1].shape[0]))
print(f"%: {((cumulative_df[cumulative_df['Injury'] == 1].shape[0] / cumulative_df.shape[0]) * 100):.2f}%")

No Injury: 4663
Injury: 37
%: 0.79%


In [39]:
# Define columns to consider

# columns_to_consider = ['TD-3', '>19.8-3', '>25-3',
#        'ACC-3', 'DEC-3', 'Sprints-3', 'Mins-3', 'TD-7', '>19.8-7', '>25-7',
#        'ACC-7', 'DEC-7', 'Sprints-7', 'Mins-7', 'TD-21', '>19.8-21', '>25-21',
#        'ACC-21', 'DEC-21', 'Sprints-21', 'Mins-21']

columns_to_consider = ['TD-7', '>19.8-7', '>25-7',
       'ACC-7', 'DEC-7', 'Sprints-7', 'Mins-7', 'TD-21', '>19.8-21', '>25-21',
       'ACC-21', 'DEC-21', 'Sprints-21', 'Mins-21']

# Count zero values in the specified columns for each row
zero_counts = (cumulative_df[columns_to_consider] == 0).sum(axis=1)

# Filter out rows with 3 or more zeros in the specified columns
cumulative_df = cumulative_df[zero_counts < 3]

In [40]:
# Count of rows containing 0 for each column
zero_counts = (cumulative_df == 0).sum()

# Creating a DataFrame for the results
result_df = pd.DataFrame({
    "Column Name": zero_counts.index,
    "Number of Rows with 0": zero_counts.values
})

result_df

,Column Name,Number of Rows with 0
0,DATE,0
1,Microcycle,0
2,Injury,4398
3,season,0
4,preseason-season,0
5,TD,0
6,>19.8,213
7,>25,1449
8,ACC,0
9,DEC,0


In [41]:
print("No Injury: " + str(cumulative_df[cumulative_df["Injury"] == 0].shape[0]))
print("Injury: " + str(cumulative_df[cumulative_df["Injury"] == 1].shape[0]))
print(f"%: {((cumulative_df[cumulative_df['Injury'] == 1].shape[0] / cumulative_df.shape[0]) * 100):.2f}%")

No Injury: 4398
Injury: 36
%: 0.81%


# Do the shift for X previous days injury

If a player got injured X day, it means that the player has been loaded and could have happened the previous session or maybe in the previous 2 sessions.
To add more Injury columns, make the previous 2 days as Injury = 1.

In [48]:
def shift_injuries_x_days(df, x):
    copy_df = df.copy()

    # Iterate over the df to shift injury labels
    for idx, row in copy_df.iterrows():
        if row['Injury'] == 1:
            # Get the player's ID and the injury date
            player_id = row['PlayerID']
            injury_date = row['DATE']

            # Shift injury labels for the current day and the previous x days
            for i in range(0, x + 1):
                prev_date = pd.to_datetime(injury_date) - pd.Timedelta(days=i)
                mask = (copy_df['PlayerID'] == player_id) & (copy_df['DATE'] == prev_date)
                copy_df.loc[mask, 'Injury'] = 1

    return copy_df

In [43]:
# df_injury_shifted = shift_injuries_x_days(cumulative_df, 2)
df_injury_shifted = cumulative_df.copy()

In [45]:
print("No Injury: " + str(df_injury_shifted[df_injury_shifted["Injury"] == 0].shape[0]))
print("Injury: " + str(df_injury_shifted[df_injury_shifted["Injury"] == 1].shape[0]))
print(f"%: {((df_injury_shifted[df_injury_shifted['Injury'] == 1].shape[0] / df_injury_shifted.shape[0]) * 100):.2f}%")

No Injury: 4398
Injury: 36
%: 0.81%


In [46]:
df_injury_shifted.head()

,DATE,Microcycle,Injury,season,preseason-season,TD,>19.8,>25,ACC,DEC,PlayerID,Player Position,Max Speed,Sprints,Mins,Max Speed Season,Avg Speed Season,% Max Speed,Speed Diff Max Avg,TD-1,>19.8-1,>25-1,ACC-1,DEC-1,Sprints-1,Mins-1,TD-3,>19.8-3,>25-3,ACC-3,DEC-3,Sprints-3,Mins-3,TD-7,>19.8-7,>25-7,ACC-7,DEC-7,Sprints-7,Mins-7,TD-21,>19.8-21,>25-21,ACC-21,DEC-21,Sprints-21,Mins-21
1,2021-08-11,M-3,0.0,2021-2022,SEASON,7423.0,426.0,20.0,52.0,38.0,23085.0,CENTRE MIDFIELDER,26.79,24.0,90.85,30.73,20.278653,87.178653,32.109364,5651.0,197.0,34.0,63.0,36.0,19.0,80.35,5651.0,197.0,34.0,63.0,36.0,19.0,80.35,5651.0,197.0,34.0,63.0,36.0,19.0,80.35,5651.0,197.0,34.0,63.0,36.0,19.0,80.35
2,2021-08-12,M-2,0.0,2021-2022,SEASON,1268.0,0.0,0.0,11.0,5.0,23085.0,CENTRE MIDFIELDER,16.57,0.0,43.58,30.73,20.278653,53.921250,-18.288460,7423.0,426.0,20.0,52.0,38.0,24.0,90.85,13074.0,623.0,54.0,115.0,74.0,43.0,171.20,13074.0,623.0,54.0,115.0,74.0,43.0,171.20,13074.0,623.0,54.0,115.0,74.0,43.0,171.20
3,2021-08-13,M-1,0.0,2021-2022,SEASON,4135.0,112.0,2.0,47.0,20.0,23085.0,CENTRE MIDFIELDER,25.47,11.0,58.35,30.73,20.278653,82.883176,25.600056,1268.0,0.0,0.0,11.0,5.0,0.0,43.58,14342.0,623.0,54.0,126.0,79.0,43.0,214.78,14342.0,623.0,54.0,126.0,79.0,43.0,214.78,14342.0,623.0,54.0,126.0,79.0,43.0,214.78
4,2021-08-15,M+1,0.0,2021-2022,SEASON,6379.0,86.0,0.0,49.0,48.0,23085.0,CENTRE MIDFIELDER,23.37,13.0,79.48,30.73,20.278653,76.049463,15.244339,0.0,0.0,0.0,0.0,0.0,0.0,0.00,5403.0,112.0,2.0,58.0,25.0,11.0,101.93,18477.0,735.0,56.0,173.0,99.0,54.0,273.13,18477.0,735.0,56.0,173.0,99.0,54.0,273.13
5,2021-08-17,M-4,0.0,2021-2022,SEASON,6287.0,209.0,9.0,62.0,44.0,23085.0,CENTRE MIDFIELDER,26.80,19.0,83.69,30.73,20.278653,87.211194,32.158677,0.0,0.0,0.0,0.0,0.0,0.0,0.00,6379.0,86.0,0.0,49.0,48.0,13.0,79.48,24856.0,821.0,56.0,222.0,147.0,67.0,352.61,24856.0,821.0,56.0,222.0,147.0,67.0,352.61


# One Hot Encoding

One Hot Encoding for Player Position

In [47]:
df_injury_shifted['Player Position'] = df_injury_shifted['Player Position'].str.replace(" ", "")
df_injury_shifted['Player Position'].unique()

# Example: One-hot encoding 'Player Position' column
encoded_final_df = pd.get_dummies(df_injury_shifted, columns=['Player Position'], prefix='Position')

columns_to_convert = [
    'Position_CENTREBACK',
    'Position_CENTREMIDFIELDER',
    'Position_FULLBACK',
    'Position_STRIKER',
    'Position_WINGER'
]

# Convert the columns from bool to int
encoded_final_df[columns_to_convert] = encoded_final_df[columns_to_convert].astype(int)

# Display the resulting dataframe
encoded_final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4434 entries, 1 to 4722
Data columns (total 51 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   DATE                       4434 non-null   datetime64[ns]
 1   Microcycle                 4236 non-null   object        
 2   Injury                     4434 non-null   float64       
 3   season                     4434 non-null   object        
 4   preseason-season           4434 non-null   object        
 5   TD                         4434 non-null   float64       
 6   >19.8                      4434 non-null   float64       
 7   >25                        4434 non-null   float64       
 8   ACC                        4434 non-null   float64       
 9   DEC                        4434 non-null   float64       
 10  PlayerID                   4434 non-null   float64       
 11  Max Speed                  4434 non-null   float64       
 12  Sprints    

# Export data

In [107]:
#encoded_final_df.to_excel('/content/drive/MyDrive/WATFORD FC/Datos GPS/datos_finales_tommy2.xlsx', index=False)

In [48]:
encoded_final_df.to_excel('data/cleaned_data_no_injured.xlsx', index=False)